In [1]:
import sys
sys.path.append("D:/Onedrive/Github/my_repos/eextract/")

import eextract
import ee

import pandas as pd

import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:1081'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:1081'

In [2]:
eextract.init_ee(project='ee-xieyx')

In [3]:
df_site_info = pd.read_excel('D:/OneDrive/Github/other_repos/BEPS.jl/examples/xieyx/data/site_info_st39.xlsx')
df_site_info = df_site_info[['site', 'lon', 'lat']]
df_site_info

,site,lon,lat
0,SanJiangYuan,100.699167,35.253056
1,LinZe,100.140833,39.327222
2,DanZhou,109.475000,19.546389
3,YuanJiang,102.177500,23.473889
4,QianYanZhou,115.066667,26.733333
5,NanDi,113.052222,29.487500
6,JuRong,119.217222,31.806667
7,HuZhong,121.017778,51.781111
8,AiLaoShan,101.028889,24.538056
9,GuCheng,115.666667,39.133333


In [4]:
ee_points = eextract.df_to_points(df_site_info)

### Tidy ERA5-Land meteorological forcing directly on GEE

In [5]:
imgcol = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')

In [6]:
def tidy_ERA5L_daily_forcing(img: ee.image.Image) -> ee.image.Image:
  time_start = ee.Date(img.get('system:time_start'))

  # temperature
  Ta = img.select(['temperature_2m'], ['Ta']).subtract(273.15)
  Tdew = img.select(['dewpoint_temperature_2m'], ['Tdew']).subtract(273.15)
  Ts = img.select(['skin_temperature'], ['Ts']).subtract(273.15)
  
  Pa = img.select(['surface_pressure'], ['Pa']).divide(1000)

  U10 = img.select(
    ["u_component_of_wind_10m", "v_component_of_wind_10m"], ['u10', 'v10']
  ).expression("sqrt(b('u10')*b('u10') + b('v10')*b('v10'))").rename('U10')
  U2 = U10.expression('b("U10") * 4.87 / log(67.8 * 10 - 5.42)').rename('U2')

  es = Ta.expression("0.6108 * exp((17.27 * b('Ta'))/(b('Ta') + 237.3))").rename('es')
  ea = Tdew.expression("0.6108 * exp((17.27 * b('Tdew'))/(b('Tdew') + 237.3))").rename('ea')
  q = ee.Image([ea, Pa]).expression(
    "0.6220016 * b('ea') / (b('Pa') - (1 - 0.6220016) * b('ea'))"
  ).rename('q')
  VPD = ee.Image([es, ea]).expression("b('es') - b('ea')").rename('VPD')

  # radiation
  Rs = img.select(['surface_solar_radiation_downwards_sum'], ['Rs']).divide(3600 * 24)
  Rns = img.select(['surface_net_solar_radiation_sum'], ['Rns']).divide(3600 * 24)
  Rn = img.select(
    ['surface_net_solar_radiation_sum', 'surface_net_thermal_radiation_sum'],
    ['Rns', 'Rnl']
  ).divide(3600 * 24).expression("b('Rns') + b('Rnl')").rename('Rn')
  # TODO: Rs_toa

  Prcp = img.select(['total_precipitation_sum'], ['Prcp']).divide(1000)
  ET_ERA5L = img.select(['total_evaporation_sum'], ['ET_ERA5L']).divide(1000).multiply(-1)
  PET_ERA5L = img.select(['potential_evaporation_sum'], ['PET_ERA5L']).divide(1000).multiply(-1)

  return ee.Image([
    Ta, Tdew, Ts, Pa, U2, q, VPD,
    Rs, Rns, Rn,
    Prcp, ET_ERA5L, PET_ERA5L
  ]).copyProperties(img, img.propertyNames()) \
    .set('date', time_start.format('yyyy-MM-dd'))

In [ ]:
for year in range(1980, 2024):
  date_start = f'{year}-01-01'
  date_end = f'{year+1}-01-01'

  imgcol_temp = imgcol.filterDate(ee.Date(date_start), ee.Date(date_end))
  imgcol_temp = imgcol_temp.map(tidy_ERA5L_daily_forcing)

  df = eextract.extract_point_imgcol(imgcol_temp, ee_points, format_date = 'yyyy-MM-dd', to_df=True, cores=16, verbose=False)
  df.to_csv(
    f'D:/Onedrive/Github/my_repos/hydroET/inst/data/ERA5L_daily_forcing_{year}.csv',
    index = False
  )

The Dataframe has 14274 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14274 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14274 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14274 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14274 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14274 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14235 rows in total.
The Dataframe has 14274 rows in total.
The Dataframe has 14235 r